# 第五章：格式化输出和为Claude预设响应

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re
import openai
import os

# 优先从环境变量读取API配置，如果没有则从IPython存储中检索
API_KEY = os.getenv('OPENAI_API_KEY')
BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
MODEL_NAME = os.getenv('OPENAI_MODEL_NAME', 'gpt-4o')

# 如果环境变量中没有API_KEY，尝试从IPython存储中获取
if not API_KEY:
    try:
        %store -r API_KEY
        %store -r MODEL_NAME
    except:
        pass

# 如果没有设置MODEL_NAME，使用默认值
if not MODEL_NAME:
    MODEL_NAME = "gpt-4o"

# 创建OpenAI客户端实例
client = openai.OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# 新增了prefill参数用于预填充文本，默认值为空字符串
def get_completion(prompt: str, system_prompt="", prefill=""):
    """
    获取GPT的完成响应
    
    参数:
        prompt (str): 用户提示内容
        system_prompt (str): 系统提示（可选）
        prefill (str): 预填充文本（可选），用于引导GPT的响应开始
    
    返回:
        str: GPT的响应文本
    """
    # 构建消息列表
    messages = []
    
    # 如果有系统提示，添加系统消息
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # 添加用户消息
    messages.append({"role": "user", "content": prompt})
    
    # 如果有预填充内容，添加助手消息
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    # 调用OpenAI API
    response = client.chat.completions.create(
        model=MODEL_NAME,              # 使用的模型名称
        messages=messages,             # 消息列表
        max_tokens=2000,              # 最大生成的token数量
        temperature=0.0               # 温度参数，0表示更确定性的回答
    )
    
    # 如果有预填充内容，将其与生成的内容组合
    if prefill:
        return prefill + response.choices[0].message.content
    else:
        return response.choices[0].message.content

---

## 课程 {#lesson}

**Claude可以以多种方式格式化其输出**。您只需要明确要求它这样做！

其中一种方式是使用XML标签将响应与其他多余文本分离。您已经学会了可以使用XML标签使您的提示对Claude更清晰、更易解析。事实证明，您还可以要求Claude**使用XML标签使其输出对人类更清晰、更容易理解**。

### 示例

还记得我们在第二章中通过要求Claude完全跳过前言来解决的"诗歌前言问题"吗？事实证明，我们也可以通过**告诉Claude将诗歌放在XML标签中**来实现类似的效果。

In [ ]:
# 可变内容变量
ANIMAL = "Rabbit"

# 带有占位符的提示模板
# 要求Claude将结果放在特定的XML标签中，便于后续处理
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

为什么我们要这样做呢？因为将输出放在**XML标签中可以让最终用户通过编写简短的程序来提取XML标签之间的内容，从而可靠地获得诗歌内容且只有诗歌内容**。

这种技术的一个扩展是**将第一个XML标签放在`assistant`轮次中。当您在`assistant`轮次中放置文本时，您基本上是在告诉Claude它已经说了一些话，应该从那个点继续。这种技术被称为"为Claude代言"或"预填充Claude的响应"。**

下面，我们用第一个`<haiku>`XML标签做了这件事。注意Claude如何直接从我们停下的地方继续。

In [ ]:
# 可变内容变量
ANIMAL = "Cat"

# 带有占位符的提示模板
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# 预填充Claude的响应
# 这个技术强制Claude从我们指定的标签开始，确保输出格式的一致性
PREFILL = "<haiku>"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次:")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claude在使用其他输出格式方面也很出色，特别是`JSON`格式。如果您想强制JSON输出（虽然不是确定性的，但接近确定性），您也可以用开括号`{`来预填充Claude的响应。

In [ ]:
# 可变内容变量
ANIMAL = "Cat"

# 带有占位符的提示模板
# 明确要求使用JSON格式并指定键名
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# 预填充Claude的响应
# 使用开括号强制Claude输出JSON格式
PREFILL = "{"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

下面是一个**在同一个提示中使用多个输入变量和输出格式规范，全部使用XML标签完成**的示例。

In [ ]:
# 第一个输入变量
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# 第二个输入变量
ADJECTIVE = "olde english"

# 带有占位符的提示模板
# 这个例子展示如何在同一个提示中使用多个变量，并用XML标签格式化输出
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# 预填充Claude的响应（现在是带有变量的f字符串）
# 使用动态XML标签名称，根据ADJECTIVE变量的值生成对应的标签
PREFILL = f"<{ADJECTIVE}_email>"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 额外课程 {#额外lesson}

如果您通过API调用Claude，可以将结束XML标签传递给`stop_sequences`参数，让Claude在输出您期望的标签后停止采样。这可以通过消除Claude在已经给出您关心的答案后的结束语来节省金钱和减少最后一个token的时间。

如果您想要在不更改上述任何内容的情况下尝试课程提示，请滚动到课程笔记本的最底部访问[**示例演练场**](#example-playground)。

---

## 练习 {#exercises}
- [练习 5.1 - 斯蒂芬·库里最佳球员](#exercise-51---steph-curry-goat)
- [练习 5.2 - 两首俳句](#exercise-52---two-haikus)
- [练习 5.3 - 两首俳句，两种动物](#exercise-53---two-haikus-two-animals)

### 练习 5.1 - 斯蒂芬·库里最佳球员 {#exercises-51-斯蒂芬库里最佳球员}
被迫做出选择时，Claude会指定迈克尔·乔丹为历史上最好的篮球运动员。我们能让Claude选择其他人吗？

修改`PREFILL`变量来**迫使Claude做出详细论证，证明历史上最好的篮球运动员是斯蒂芬·库里**。尽量不要改变除了`PREFILL`之外的任何内容，因为这是本练习的重点。

In [ ]:
# 带有占位符的提示模板
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# 预填充Claude的响应（请编辑这个！）
# 提示：通过预填充特定的开头，可以引导Claude朝特定方向回答
# 例如："在我看来，斯蒂芬·库里是历史上最好的篮球运动员，因为..."
PREFILL = ""

# 获取Claude的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 检查练习答案是否正确
# 检查响应中是否包含"Warrior"（勇士队），这暗示提到了库里
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 练习 5.2 - 两首俳句 {#exercises-52-two-haikus}
使用XML标签修改下面的`PROMPT`，使Claude写两首关于动物的俳句而不是只写一首。应该清楚地看出一首诗在哪里结束，另一首在哪里开始。

In [ ]:
# 可变内容变量
ANIMAL = "cats"

# 带有占位符的提示模板
# 需要修改这个提示来生成两首俳句，并用XML标签分隔
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# 预填充Claude的响应
PREFILL = "<haiku>"

# 获取Claude的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 检查练习答案是否正确
# 检查响应是否包含"cat"和"<haiku>"标签，并且至少有5行
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 练习 5.3 - 两首俳句，两种动物 {#exercises-53-two-haikus两种动物}
修改下面的`PROMPT`，使**Claude生成两首关于两种不同动物的俳句**。使用`{ANIMAL1}`作为第一个替换的占位符，使用`{ANIMAL2}`作为第二个替换的占位符。

In [ ]:
# 第一个输入变量
ANIMAL1 = "Cat"

# 第二个输入变量
ANIMAL2 = "Dog"

# 带有占位符的提示模板
# 需要修改这个提示来使用两个动物变量并生成两首俳句
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# 获取Claude的响应
response = get_completion(PROMPT)

# 检查练习答案是否正确
# 检查响应是否包含"tail"、"cat"和"<haiku>"标签
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(response)
print("\n---------------==================== 练习评分 ====================---------------")
result = grade_exercise(response)
print("✅ 练习完成状态:", "正确 ✅" if result else "需要调整 ❌")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示工程愉快！

---

## 示例演练场 {#example-playground}

这是一个供您自由实验本课程中展示的提示示例并调整提示来观察对Claude响应影响的区域。

In [ ]:
# 可变内容
ANIMAL = "Rabbit"

# 带有占位符的提示模板
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
ANIMAL = "Cat"

# 带有占位符的提示模板
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# 预填充Claude的响应
PREFILL = "<haiku>"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次:")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 可变内容
ANIMAL = "Cat"

# 带有占位符的提示模板
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# 预填充Claude的响应
PREFILL = "{"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 第一个输入变量
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# 第二个输入变量
ADJECTIVE = "olde english"

# 带有占位符的提示模板
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# 预填充Claude的响应（现在是带有变量的f字符串）
PREFILL = f"<{ADJECTIVE}_email>"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))